# BigDataViewer in imglyb

In [1]:
import h5py
import numpy as np
import os
import pathlib
import skimage.io
import subprocess
import tempfile
import timeit
import threading
import time

## imglyb imports
An additional *tmp* directory within the classpath will allow us to compile Java code on the fly if we need fast pixel-wise method calls (see example below). 

In [2]:
tmp_dir = tempfile.mkdtemp()

import jnius_config
jnius_config.add_classpath( tmp_dir )

import imglyb
import imglyb.util as util

from jnius import autoclass, PythonJavaClass, java_method, cast

## Wait For BDV
Python is not aware of any Java threads and -- if not run in interactive mode -- would exit even with an active BDV window open. The `bdv_exit_check` as defined below keeps a Python thread alive while BDV is running and prevents Python from exiting. This is not necessary for interactive sessions (e.g. IPython shell, notebook) but is still added for the purpose of demonstration.

In [3]:
def bdv_exit_check( viewer, run_on_exit = lambda : print( "Exited!" ) ):

    check = autoclass( 'net.imglib2.python.BdvWindowClosedCheck' )()
    frame = cast( 'javax.swing.JFrame', autoclass( 'javax.swing.SwingUtilities' ).getWindowAncestor( viewer ) )
    frame.addWindowListener( check )

    def sleeper():
        while check.isOpen():
            time.sleep( 0.1 )
        run_on_exit()

    t = threading.Thread( target=sleeper )
    t.start()

## Multi-Channel vs ARGB

In [4]:
url = 'http://www.nerdtests.com/mq/testimages/167138_4f49b66c0cb4a87cc906.jpg'
arr = skimage.io.imread( url )
rai = imglyb.to_imglib( arr )
bdv = util.BdvFunctions.show( rai, 'argb' )
bdv_exit_check( bdv.getBdvHandle().getViewerPanel() )

Exited!


In [5]:
rai = imglyb.to_imglib( np.transpose( arr, ( 2, 0, 1 ) ) )
bdv = util.BdvFunctions.show( rai, 'argb', util.BdvOptions.options().is2D() )
bdv_exit_check( bdv.getBdvHandle(). getViewerPanel() )

Exited!


## Pure Python Converter

In [6]:
ARGBType = autoclass( 'net.imglib2.type.numeric.ARGBType' )

class CompositeARGBConverter( PythonJavaClass ):
    
    __javainterfaces__ = [ 'net.imglib2.converter.Converter' ]
    
    def __init__( self ):
        super( CompositeARGBConverter, self ).__init__()
        
    @java_method( '(Ljava/lang/Object;Ljava/lang/Object;)V' )
    def convert( self, source, target ):
        target.set( ARGBType.rgba( 
            source.get( 0 ).getInteger(), 
            source.get( 1 ).getInteger(), 
            source.get( 2 ).getInteger(),
            255 ) )
        
       
Converters = autoclass( 'net.imglib2.converter.Converters' )
converter = CompositeARGBConverter()
composite = util.Views.collapseReal( rai )
view = Converters.convert( cast( 'net.imglib2.RandomAccessibleInterval', composite ), converter, ARGBType() )
img = autoclass( 'net.imglib2.img.array.ArrayImgs' ).argbs( view.dimension( 0 ), view.dimension( 1 ) )

start = timeit.default_timer()
util.Helpers.burnIn( view, img )
stop = timeit.default_timer()
print( 'Runtime for converting every pixel:', stop - start )
print( 'Image size:', arr.shape )

Runtime for converting every pixel: 22.50061784999707
Image size: (282, 330, 3)


## Pure Java Converter

In [7]:
converter_code = """
import net.imglib2.converter.Converter;
import net.imglib2.type.numeric.ARGBType;
import net.imglib2.type.numeric.integer.UnsignedByteType;
import net.imglib2.view.composite.Composite;

public class CompositeARGBTypeConverter< C extends Composite< UnsignedByteType > > implements Converter< C, ARGBType > {
    
    public void convert( C source, ARGBType target ) {
        target.set( ARGBType.rgba( source.get( 0 ).get(), source.get( 1 ).get(), source.get( 2 ).get(), 255 ) );
    }
    
}
"""

fp = pathlib.Path( tmp_dir ) / 'CompositeARGBTypeConverter.java'
print( tmp_dir )
with open( fp, 'w' ) as f:
    f.write( converter_code )

javac = pathlib.Path( os.environ[ 'JAVA_HOME' ] ) / 'bin' / 'javac'
proc = subprocess.run( 
    [ javac, '-cp', jnius_config.split_char.join( jnius_config.get_classpath() ), fp ],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE)
if proc.returncode != 0:
    print ( proc.stderr )

/tmp/tmpjbb8onlw


In [8]:
Converters = autoclass( 'net.imglib2.converter.Converters' )
converter = autoclass( 'CompositeARGBTypeConverter' )()
composite = util.Views.collapseReal( rai )
view = Converters.convert( cast( 'net.imglib2.RandomAccessibleInterval', composite ), converter, ARGBType() )
img = autoclass( 'net.imglib2.img.array.ArrayImgs' ).argbs( view.dimension( 0 ), view.dimension( 1 ) )

start = timeit.default_timer()
util.Helpers.burnIn( view, img )
stop = timeit.default_timer()
print( 'Runtime for converting every pixel:', stop - start )
print( 'Image size:', arr.shape )

Runtime for converting every pixel: 0.030213366997486446
Image size: (282, 330, 3)


In [9]:
bdv = util.BdvFunctions.show( view, 'argb', util.BdvOptions.options().is2D() )
bdv_exit_check( bdv.getBdvHandle().getViewerPanel(), lambda : print( 'Different exit message!' ) )

Different exit message!


In [10]:
path = '/home/phil/local/tmp/some-file.h5'
with h5py.File( path, 'r' ) as f:
    data = f['some/data'][ :300, :200, :100 ]
argb = imglyb.to_imglib_argb( data )
bdv = util.BdvFunctions.show( argb, 'argb' )
bdv_exit_check( bdv.getBdvHandle().getViewerPanel() )

Exited!
